# TIC TAC TOE 

In [74]:
import numpy as np
import pickle

BOARD_ROWS = 3
BOARD_COLS = 3
ALIGN = 3

## Classe State

Nous créons une classe State : elle permet d'enregistrer l'état de la grille de jeu, de la mettre à jour lorsqu'un joueur joue, de l'afficher, de décréter si la partie est finie et de récompenser les joueurs en conséquence. 

In [233]:
class State :
    
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        self.playerSymbol = 1
        
    """
On initialise une grille 3x3 remplie de 0 et deux joueurs p1 et p2. 
L'attribut playSymbol permet de matérialiser le tour d'un joueur : il vaut 1 lorsque c'est à p1 de jouer 
et -1 lorsque c'est à p2. Par défaut, p1 joue en premier. 
Dès qu'un joueur effectuera une action, playSymbol changera de valeur. 
    """
        
    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
        return self.boardHash

    """
La méthode getHash hache l'état de la grille.
    """ 

    def showBoard(self):
        # p1 : x  p2 : o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')

    """
La méthode showBoard permet d'afficher l'interface du jeu. 
    """    
    
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # tuple : coordonnées des cases vides
        return positions

    """
La méthode availablePositions permet à chaque tour de jeu de garder en mémoire la liste des positions des cases 
vides, afin de mettre à jour la grille pour le tour suivant.
    """
    
    
    def updateState(self, position):
        self.board[position] = self.playerSymbol
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1

    """
Lorsqu'un joueur effectue une action, son symbole correspondant (1 pour X et -1 pour O) remplacera 0 dans la grille, 
et playSymbol changera de valeur afin de passer à l'autre joueur. 
    """
    
    def winner(self):
        # si trois symboles sont alignés en ligne
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == ALIGN :
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == - ALIGN :
                self.isEnd = True
                return -1
        # si trois symboles sont alignés en colonne
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == ALIGN :
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == - ALIGN :
                self.isEnd = True
                return -1
        # si trois symboles sont alignés en diagonale
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)]) # deux types de diagonale
        diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
        diag_sum = max(abs(diag_sum1), abs(diag_sum2))
        if diag_sum == ALIGN :
            self.isEnd = True
            if diag_sum1 == ALIGN or diag_sum2 == ALIGN :
                return 1
            else:
                return -1

        # si égalité
        if len(self.availablePositions()) == 0 : # plus de position disponible --> jeu fini
            self.isEnd = True 
            return 0
        
        # jeu pas fini
        self.isEnd = False
        return None
    
    """
Après chaque action effectuée par le joueur, la méthode winner vérifie en continu si le jeu est terminé et si oui, 
juge le vainqueur du jeu et récompense les deux joueurs.
Pour ceci, on vérifie la somme des lignes, des colonnes et des diagonales, et retourne 1 si p1 gagne, 
-1 si p2 gagne, 0 si nul et None si le jeu n'est pas encore terminé.
On met également à jour la valeur de l'attribut isEnd.
    """

    def giveReward(self):
        result = self.winner()
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0)
            self.p2.feedReward(0)
    """
À la fin de la partie, on attribue 1 au gagnant et 0 au perdant. On fait appel à la méthode feedReward 
qui appartient à la classe Player.
On considère qu'un match nul est une mauvaise fin. En cas d'égalité, on donne donc 0 comme récompense aux joueurs. 
    """

  
    def play(self, rounds=400000):
        for i in range(rounds):
            if i % 10000 == 0 :
                print("Rounds {}".format(i))
            begin = np.random.randint(0,2) # on génère aléatoirement 0 ou 1 pour savoir si p2 ou p1 commence
            if begin == 0 : # p1 commence 
                while not self.isEnd :
                    # Player 1
                    positions = self.availablePositions()
                    p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol) # choix d'une action
                    self.updateState(p1_action) # on update la grille
                    board_hash = self.getHash()
                    self.p1.addState(board_hash) # on update le hashage de la grille dans le dict états-valeurs
                
                    # pour ajouter les états non visités symétriques ou rotatoires à celui-ci
                    if BOARD_ROWS == BOARD_ROWS : # cas où la grille de jeu est carrée
                        # rotation de 90 degrés à droite
                        self.p1.addState(str(np.rot90(self.board).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # rotation de 90 degrés à gauche
                        self.p1.addState(str(np.rot90(self.board,3).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # rotation de 180 degrés
                        self.p1.addState(str(np.rot90(self.board,2).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # symétrie verticale
                        self.p1.addState(str(np.flip(self.board, 1).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # symétrie horizontale
                        self.p1.addState(str(np.flip(self.board, 0).reshape(BOARD_COLS * BOARD_ROWS))) 

                
                    win = self.winner() # on regarde si le jeu est fini 
                    if win is not None :
                        self.giveReward() 
                        self.p1.reset() # permet de réinitialiser la liste des états des joueurs
                        self.p2.reset()
                        self.reset()
                        break

                    else : 
                        # Player 2
                        positions = self.availablePositions() # actions possibles
                        p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol) # choix d'une action
                        self.updateState(p2_action) # on update la grille
                        board_hash = self.getHash()
                        self.p2.addState(board_hash) # on update le hashage de la grille dans le dict états-valeurs
                    
                        # pour ajouter les états non visités symétriques ou rotatoires à celui-ci
                        if BOARD_ROWS == BOARD_ROWS : # cas où la grille de jeu est carrée
                            # rotation de 90 degrés à droite
                            self.p2.addState(str(np.rot90(self.board).reshape(BOARD_COLS * BOARD_ROWS)))
                            # rotation de 90 degrés à gauche
                            self.p2.addState(str(np.rot90(self.board,3).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # rotation de 180 degrés
                            self.p2.addState(str(np.rot90(self.board,2).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # symétrie verticale
                            self.p2.addState(str(np.flip(self.board, 1).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # symétrie horizontale
                            self.p2.addState(str(np.flip(self.board, 0).reshape(BOARD_COLS * BOARD_ROWS)))                    
                                     
                        win = self.winner() # on regarde si le jeu est fini 
                        if win is not None :
                            self.giveReward() 
                            self.p1.reset() # permet de réinitialiser la liste des états des joueurs
                            self.p2.reset()
                            self.reset()
                            break
                            
                            
            else : # p2 commence
                while not self.isEnd :
                    # Player 2
                    playerSymbol = -1
                    positions = self.availablePositions() # actions possibles
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol) # choix d'une action
                    self.updateState(p2_action) # on update la grille
                    board_hash = self.getHash()
                    self.p2.addState(board_hash) # on update le hashage de la grille dans le dict états-valeurs
                    
                    # pour ajouter les états non visités symétriques ou rotatoires à celui-ci
                    if BOARD_ROWS == BOARD_ROWS : # cas où la grille de jeu est carrée
                        # rotation de 90 degrés à droite
                        self.p2.addState(str(np.rot90(self.board).reshape(BOARD_COLS * BOARD_ROWS)))
                        # rotation de 90 degrés à gauche
                        self.p2.addState(str(np.rot90(self.board,3).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # rotation de 180 degrés
                        self.p2.addState(str(np.rot90(self.board,2).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # symétrie verticale
                        self.p2.addState(str(np.flip(self.board, 1).reshape(BOARD_COLS * BOARD_ROWS))) 
                        # symétrie horizontale
                        self.p2.addState(str(np.flip(self.board, 0).reshape(BOARD_COLS * BOARD_ROWS)))                    
                                 
                    win = self.winner() # on regarde si le jeu est fini 
                    if win is not None :
                        self.giveReward() 
                        self.p1.reset() # permet de réinitialiser la liste des états des joueurs
                        self.p2.reset()
                        self.reset()
                        break
                        
                        
                        
                    else : 
                        # Player 1
                        positions = self.availablePositions()
                        p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol) # choix d'une action
                        self.updateState(p1_action) # on update la grille
                        board_hash = self.getHash()
                        self.p1.addState(board_hash) # on update le hashage de la grille dans le dict états-valeurs
                
                        # pour ajouter les états non visités symétriques ou rotatoires à celui-ci
                        if BOARD_ROWS == BOARD_ROWS : # cas où la grille de jeu est carrée
                            # rotation de 90 degrés à droite
                            self.p1.addState(str(np.rot90(self.board).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # rotation de 90 degrés à gauche
                            self.p1.addState(str(np.rot90(self.board,3).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # rotation de 180 degrés
                            self.p1.addState(str(np.rot90(self.board,2).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # symétrie verticale
                            self.p1.addState(str(np.flip(self.board, 1).reshape(BOARD_COLS * BOARD_ROWS))) 
                            # symétrie horizontale
                            self.p1.addState(str(np.flip(self.board, 0).reshape(BOARD_COLS * BOARD_ROWS))) 

                
                        win = self.winner() # on regarde si le jeu est fini 
                        if win is not None :
                            self.giveReward() 
                            self.p1.reset() # permet de réinitialiser la liste des états des joueurs
                            self.p2.reset()
                            self.reset()
                            break

                    

    """
Pour que l'agent apprenne, on le laisse jouer contre un autre agent (on peut choisir le nombre de rounds 
mais par défaut, ils jouent 400 000 fois). p1 ou p2 commence aléatoirement. 
Pendant l'entraînement, le processus pour chaque joueur est :
 - Rechercher les positions possibles
 - Choisir une action
 - Mettre à jour l'état de la grille de jeu et ajouter l'action aux états du joueur
Si un joueur atteint la fin du jeu, il sera récompensé en conséquence. 
Pour ceci, on fait appel aux méthodes de la classe Player. 
    """
                        

    def play2(self, begin = 1):
        if begin == 1 : # Player 1 puis Player 2
             while not self.isEnd :
                # Player 1
                positions = self.availablePositions() # actions possibles
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol) # choix d'une action
                self.updateState(p1_action) # on update la grille
                self.showBoard()
            
                win = self.winner() # on regarde si le jeu est fini
                if win is not None :
                    if win == 1 : # p1 a gagné
                        print(self.p1.name, " a gagné !")
                    else : # match nul
                        print("Egalité !")
                    self.reset() # on reset la grille de jeu
                    break

                else :
                    # Player 2
                    positions = self.availablePositions()
                    p2_action = self.p2.chooseAction(positions)  # choix d'une action parmi les actions possibles
                    self.updateState(p2_action)  # on update la grille
                    self.showBoard()
                
                    win = self.winner() # on regarde si le jeu est fini
                    if win is not None :
                        if win == -1 : # p2 a gagné 
                            print(self.p2.name, "a gagné !")
                        else : # match nul
                            print("Egalité !")
                        self.reset() # on reset la grille de jeu
                        break
            
        else : # Player 2 puis Player 1
                while not self.isEnd :
                    # Player 2
                    playerSymbol = -1 # on change le tour
                    positions = self.availablePositions() # actions possibles
                    p2_action = self.p2.chooseAction(positions) # choix d'une action
                    self.updateState(p2_action) # on update la grille
                    self.showBoard()
            
                    win = self.winner() # on regarde si le jeu est fini 
                    if win is not None :
                        if win == 1 : # p2 a gagné
                            print(self.p2.name, " a gagné !")
                        else : # match nul
                            print("Egalité !")
                        self.reset() # on reset la grille de jeu
                        break

                    else :
                        # Player 1
                        positions = self.availablePositions()
                        p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)  # choix d'une action parmi les actions possibles
                        self.updateState(p1_action)  # on update la grille
                        self.showBoard()
                
                        win = self.winner() # on regarde si le jeu est fini 
                        if win is not None :
                            if win == -1 : # p1 a gagné 
                                print(self.p1.name, "a gagné !")
                            else : # match nul
                                print("Egalité !")
                            self.reset() # on reset la grille de jeu
                            break
                                  
                
                
    """
Une fois que l'agent a appris, on peut le faire jouer contre un humain. Pour ceci, on fait appel aux méthodes de 
la classe HumanPlayer. 
Par défaut, p1 commencer, mais en changeant la valeur du paramètre begin, on peut faire commencer p2. 
    """
    
    
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
        
    """
Quand le jeu est fini, on reset la grille de jeu. 
    """

## Classe Player

Nous créons une classe Player qui permettra à notre agent d'apprendre et d'affiner sa politique en jouant contre un autre agent. On choisira des actions basées sur l'estimation actuelle des états, on enregistrera tous les états du jeu en mettant à jour l'estimation de la valeur des états après chaque partie, pour ensuite enregistrer la politique et la charger pour l'utiliser. 


En termes de choix d'actions, nous utilisons une méthode ϵ-greedy pour équilibrer l'exploration et l'exploitation. 
Ici, nous prendrons ϵ = 0.3, ce qui signifie que 70% du temps, notre agent prendra une action basée sur l'estimation actuelle de la valeur des états (exploitation), et 30% du temps, notre agent prendra une action aléatoire (exploration). 


Pour mettre à jour l'estimation de la valeur des états, avec un taux d'apprentissage de 0.2 par défaut, nous appliquerons la règle :
la valeur actualisée de l'état t est égale à la valeur actuelle de l'état t en ajoutant la différence entre la valeur de l'état suivant et la valeur de l'état actuel, multipliée par un taux α = 0.9 par défaut. 

In [234]:
class Player :
    def __init__(self, name, learning_rate = 0.2, epsilon=0.3, alpha = 0.9):
        self.name = name
        self.states = []
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.alpha = alpha
        self.states_value = {}  # état -> valeur
    
    """     
On garde une trace de toutes les positions prises par le joueur au cours de chaque partie dans une liste self.states.
On mettra à jour les états correspondants dans le dictionnaire self.states_value (contient les paires états-valeurs). 
    """

    def getHash(self, board):
        boardHash = str(board.reshape(BOARD_COLS * BOARD_ROWS))
        return boardHash

    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.epsilon :
            idx = np.random.choice(len(positions)) # on prend une action au hasard parmi les actions posibles
            action = positions[idx] 
        else:
            value_max = -999
            for p in positions :
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value=0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                if value >= value_max :
                    value_max = value
                    action = p
        return action
        
    """
On utilise une méthode ϵ-greedy pour choisir nos actions. 
        
    """
    
    def addState(self, state):
        self.states.append(state)
        
    """
On stocke le hashage de l'état de la grille dans le dict de la valeur de l'état, 
et pendant l'exploitation, on hashe le prochain état de la grille et on choisit 
l'action qui renvoie la valeur maximale de l'état suivant.
    """

    def feedReward(self, reward) :
        for st in reversed(self.states): # on parcourt les états dans l'ordre inverse
            if self.states_value.get(st) is None :
                self.states_value[st] = 0 # par défaut la valeur d'un état est 0 
            self.states_value[st] += self.learning_rate * (self.alpha * reward - self.states_value[st]) # algo SARSA
            reward = self.states_value[st]
            
    """
A la fin du jeu, on met à jour la valeur actuelle en fonction de notre dernière observation.
    """
    
    def reset(self) :
        self.states = [] # permet de recommencer à 0

        
    def savePolicy(self) :
        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()
        
    """       
Après s'être entrainé en jouant un certain nombre de tours, l'agent est en mesure d'apprendre sa politique, stockée
dans le dict états-valeurs. On sauvegarde cette politique (par la méthode savePolicy) et on la chargera pour jouer 
contre un joueur humain (par la méthode loadPolicy).
    """
    
    def loadPolicy(self, file):
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()

## Classe HumanPlayer

Maintenant, notre agent est prêt. Nous créeons une classe HumanPlayer pour jouer nous-même contre l'agent.

In [235]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name

    def chooseAction(self, positions):
        while True:
            col = int(input("Entrez la colonne de jeu :")) 
            row = int(input("Entrez la ligne de jeu :"))
            action = (row, col)
            if action in positions :
                return action
    """
Le joueur humain choisira ses actions. On vérifie que l'action entrée par le joueur est possible, sinon on lui 
redemande.
    """

## Play

On va désormais jouer. On crée deux agents afin d'apprendre la politique. 

In [236]:
p1 = Player("p1")
p2 = Player("p2")

st = State(p1, p2)
st.play()

Rounds 0
Rounds 10000
Rounds 20000
Rounds 30000
Rounds 40000
Rounds 50000
Rounds 60000
Rounds 70000
Rounds 80000
Rounds 90000
Rounds 100000
Rounds 110000
Rounds 120000
Rounds 130000
Rounds 140000
Rounds 150000
Rounds 160000
Rounds 170000
Rounds 180000
Rounds 190000
Rounds 200000
Rounds 210000
Rounds 220000
Rounds 230000
Rounds 240000
Rounds 250000
Rounds 260000
Rounds 270000
Rounds 280000
Rounds 290000
Rounds 300000
Rounds 310000
Rounds 320000
Rounds 330000
Rounds 340000
Rounds 350000
Rounds 360000
Rounds 370000
Rounds 380000
Rounds 390000


On sauvegarde la politique des deux agents. 

In [237]:
p1.savePolicy()
p2.savePolicy()

On va désormais jouer contre p1. On charge sa politique. 

In [242]:
p1 = Player("Computer")
p1.loadPolicy("policy_p1")

p2 = HumanPlayer("Salomé")

st = State(p1, p2)
st.play2()

-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
Entrez la colonne de jeu :0
Entrez la ligne de jeu :0
-------------
| o |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
| o |   |   | 
-------------
| x | x |   | 
-------------
|   |   |   | 
-------------
Entrez la colonne de jeu :2
Entrez la ligne de jeu :1
-------------
| o |   |   | 
-------------
| x | x | o | 
-------------
|   |   |   | 
-------------
-------------
| o |   | x | 
-------------
| x | x | o | 
-------------
|   |   |   | 
-------------
Entrez la colonne de jeu :0
Entrez la ligne de jeu :2
-------------
| o |   | x | 
-------------
| x | x | o | 
-------------
| o |   |   | 
-------------
-------------
| o |   | x | 
-------------
| x | x | o | 
-------------
| o | x |   | 
-------------
Entrez la colonne de jeu :1
Entrez la ligne de jeu :0
-------------
| o | o | x | 
-------------
| x | x | o | 
-------------
| o |

In [243]:
st.play2()

-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
Entrez la colonne de jeu :2
Entrez la ligne de jeu :2
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   | o | 
-------------
-------------
|   |   |   | 
-------------
|   | x | x | 
-------------
|   |   | o | 
-------------
Entrez la colonne de jeu :0
Entrez la ligne de jeu :1
-------------
|   |   |   | 
-------------
| o | x | x | 
-------------
|   |   | o | 
-------------
-------------
|   |   |   | 
-------------
| o | x | x | 
-------------
| x |   | o | 
-------------
Entrez la colonne de jeu :2
Entrez la ligne de jeu :0
-------------
|   |   | o | 
-------------
| o | x | x | 
-------------
| x |   | o | 
-------------
-------------
|   | x | o | 
-------------
| o | x | x | 
-------------
| x |   | o | 
-------------
Entrez la colonne de jeu :0
Entrez la ligne de jeu :0
-------------
| o | x | o | 
-------------
| o | x | x | 
-------------
| x |